# HW4: implementing item-based CF with cosine
First,run recommenderDemo.ipynb and be familar with the code and data.
Second,implement item-based CF with cosine

In [7]:
from collections import defaultdict
import pandas as pd
import numpy as np
import scipy.sparse
import math
import sys

In [8]:
# From https://web.cs.wpi.edu/~kmlee/cs547/amazon_reviews_us_Musical_Instruments_v1_00_small.tsv.gz
input_file_stream = open(file="./amazon_reviews_us_Musical_Instruments_v1_00_small.tsv",mode='rt',encoding="utf8")

header = input_file_stream.readline().strip().split('\t')
print(header)

small_dataset =[]
for line1 in input_file_stream:
    fields = line1.strip().split('\t')
    d = dict(zip(header,fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    small_dataset.append(d)
print(len(small_dataset))
small_dataset_df = pd.DataFrame(small_dataset)
print(small_dataset_df.shape)

['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
20000
(20000, 15)


In [9]:
usersPerItem = defaultdict(set)
reviewsPerUser = defaultdict(list)

for d in small_dataset:
    user,item = d['customer_id'],d['product_id']
    usersPerItem[item].add(user)
    reviewsPerUser[user].append(d)

In [10]:
small_dataset_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,45610553,RMDCHWD0Y5OZ9,B00HH62VB6,618218723,AGPtek® 10 Isolated Output 9V 12V 18V Guitar P...,Musical Instruments,3,0,1,N,N,Three Stars,"Works very good, but induces ALOT of noise.",2015-08-31
1,US,14640079,RZSL0BALIYUNU,B003LRN53I,986692292,Sennheiser HD203 Closed-Back DJ Headphones,Musical Instruments,5,0,0,N,Y,Five Stars,Nice headphones at a reasonable price.,2015-08-31
2,US,6111003,RIZR67JKUDBI0,B0006VMBHI,603261968,AudioQuest LP record clean brush,Musical Instruments,3,0,1,N,Y,Three Stars,removes dust. does not clean,2015-08-31
3,US,1546619,R27HL570VNL85F,B002B55TRG,575084461,Hohner Inc. 560BX-BF Special Twenty Harmonica,Musical Instruments,5,0,0,N,Y,I purchase these for a friend in return for pl...,I purchase these for a friend in return for pl...,2015-08-31
4,US,12222213,R34EBU9QDWJ1GD,B00N1YPXW2,165236328,Blue Yeti USB Microphone - Blackout Edition,Musical Instruments,5,0,0,N,Y,Five Stars,This is an awesome mic!,2015-08-31


In [11]:
for c_name in small_dataset_df.columns:
    unique_val = small_dataset_df[c_name].nunique()
    if unique_val<11:
        print(f'\n{c_name} : Unique Values -> {unique_val},Values -> {small_dataset_df[c_name].unique()}')
    else:
        print(f'\n{c_name} : Unique Values -> {unique_val}')


marketplace : Unique Values -> 1,Values -> ['US']

customer_id : Unique Values -> 15342

review_id : Unique Values -> 20000

product_id : Unique Values -> 11182

product_parent : Unique Values -> 10099

product_title : Unique Values -> 10091

product_category : Unique Values -> 1,Values -> ['Musical Instruments']

star_rating : Unique Values -> 5,Values -> [3 5 2 4 1]

helpful_votes : Unique Values -> 60

total_votes : Unique Values -> 60

vine : Unique Values -> 2,Values -> ['N' 'Y']

verified_purchase : Unique Values -> 2,Values -> ['N' 'Y']

review_headline : Unique Values -> 11910

review_body : Unique Values -> 18475

review_date : Unique Values -> 21


In [12]:
test_matrix = small_dataset_df.pivot_table(index='product_id',columns='customer_id',values='star_rating').fillna(0)
print(f'{test_matrix.shape = }')

test_matrix.shape = (11182, 15342)


In [13]:
# https://stackoverflow.com/questions/5194057/better-way-to-convert-file-sizes-in-python
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B","KB","MB","GB","TB","PB","EB","ZB","YB")
    i = int(math.floor(math.log(size_bytes,1024)))
    p = math.pow(1024,i)
    s = round(size_bytes / p,2)
    return "%s %s" % (s,size_name[i])

In [14]:
print(convert_size(size_bytes=sys.getsizeof(test_matrix)))

1.28 GB


In [15]:
ratingMean = sum(small_dataset_df['star_rating'].values) / len(small_dataset_df)
alwaysPredictMean = [ratingMean]*len(small_dataset_df)
print(f'{ratingMean = },{alwaysPredictMean[0:5] = }')

labels = small_dataset_df['star_rating'].values.tolist()
assert small_dataset_df['star_rating'].head().tolist() == labels[0:5]
print(f'{labels[0:5] = }')

ratingMean = 4.2872,alwaysPredictMean[0:5] = [4.2872, 4.2872, 4.2872, 4.2872, 4.2872]
labels[0:5] = [3, 5, 3, 5, 5]


In [ ]:
def MSE(predictions,labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

def Cosine(f1,f2):
    f1,f2=np.array(f1),np.array(f2)
    for a in range(len(f1)):
        if f1[a]!=0:
            f1[a]=f1[a]-np.mean(f1)
        if f2[a]!=0:
              f2[a]=f2[a]-np.mean(f2)
    f3 = f1*f1
    f4 = f2*f2
    return np.round(np.dot(f1,f2)/(np.sqrt((f3).sum())*np.sqrt((f4).sum())),6)

def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['product_id']
        if i2 == item: continue
        ratings.append(d['star_rating'])
        similarities.append(Cosine(test_matrix.loc[item].values,test_matrix.loc[i2].values))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return round((sum(weightedRatings) / sum(similarities)),6)
    else:
        # User hasn't rated any similar items
        return ratingMean

cfPredictions = [predictRating(d['customer_id'],d['product_id']) for d in small_dataset]
print(MSE(alwaysPredictMean,labels))
print(MSE(cfPredictions,labels))

1.510626389999875
1.5766203176786246


(optional/bonus task: you will get additional 25 points)
download https://web.cs.wpi.edu/~kmlee/cs547/amazon_reviews_us_Musical_Instruments_v1_00_large.tsv.gz
this dataset contains over 900K user-item reviews. repeat the above process (i.e.,meauring MSE with cosine). report the MSE and compare it with MSE of alwaysPredictMean. This optional task would require better data structure and implementation.

In [17]:
input_file_stream = open(file="./amazon_reviews_us_Musical_Instruments_v1_00_large.tsv",mode='rt',encoding="utf8")

header = input_file_stream.readline().strip().split('\t')
print(header)

large_dataset =[]
for line1 in input_file_stream:
    fields = line1.strip().split('\t')
    d = dict(zip(header,fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    large_dataset.append(d)
print(len(large_dataset))
large_dataset_df = pd.DataFrame(large_dataset)
print(large_dataset_df.shape)

['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
904765
(904765, 15)


In [19]:
for c_name in large_dataset_df.columns:
    unique_val = large_dataset_df[c_name].nunique()
    if unique_val<11:
        print(f'\n{c_name} : Unique Values -> {unique_val},Values -> {large_dataset_df[c_name].unique()}')
    else:
        print(f'\n{c_name} : Unique Values -> {unique_val}')


marketplace : Unique Values -> 1,Values -> ['US']

customer_id : Unique Values -> 573149

review_id : Unique Values -> 904765

product_id : Unique Values -> 123328

product_parent : Unique Values -> 107163

product_title : Unique Values -> 110981

product_category : Unique Values -> 1,Values -> ['Musical Instruments']

star_rating : Unique Values -> 5,Values -> [3 5 2 4 1]

helpful_votes : Unique Values -> 414

total_votes : Unique Values -> 432

vine : Unique Values -> 2,Values -> ['N' 'Y']

verified_purchase : Unique Values -> 2,Values -> ['N' 'Y']

review_headline : Unique Values -> 529954

review_body : Unique Values -> 851789

review_date : Unique Values -> 4558


In [18]:
ratingMean = sum(large_dataset_df['star_rating'].values) / len(large_dataset_df)
alwaysPredictMean = [ratingMean]*len(large_dataset_df)
print(f'{ratingMean = },{alwaysPredictMean[0:5] = }')

labels = large_dataset_df['star_rating'].values.tolist()
assert large_dataset_df['star_rating'].head().tolist() == labels[0:5]
print(f'{labels[0:5] = }')

reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
for d in large_dataset:
    user,item = d['customer_id'],d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

ratingMean = 4.251102772543146,alwaysPredictMean[0:5] = [4.251102772543146, 4.251102772543146, 4.251102772543146, 4.251102772543146, 4.251102772543146]
labels[0:5] = [3, 5, 3, 5, 5]


In [ ]:
def predictRatingCosine(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['product_id']
        if i2 == item:
            continue
        if i2 in test_matrix.indices and user in test_matrix.indptr:
            ratings.append(float(test_matrix[i2,user]))
            similarities.append(Cosine(test_matrix[i2].toarray()[0].astype(float),test_matrix[item].toarray()[0].astype(float)))
    if sum(similarities) > 0:
        weightedRatings = [(x * y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

cfPredictions = [predictRatingCosine(d['customer_id'],d['product_id']) for d in large_dataset]
print(MSE(cfPredictions,large_dataset_df['star_rating']))

1.528524000000084


*-----------------
# Done

All set!

** What do you need to submit?**

* **hw4.ipynb Notebook File**: Save this Jupyter notebook with all output,and find the notebook file in your folder (for example,"filename.ipynb"). This is the file you need to submit.

** How to submit: **
        Please submit through canvas.wpi.edu
